# Graphs

While weave looks procedural, it uses a data graph backend.

In [1]:
import axiomic as ax

(axiomic) weave in helpful console mode (export AXIOMIC_CONSOLE_ENABLE=False to disable).

(axiomic) discovered OPENAI_API_KEY, models available under models.OpenAI

(axiomic) discovered ANTHROPIC_API_KEY, models available under models.Anthropic

(axiomic) discovered TOGETHER_API_KEY, models available under models.Together

In [2]:
whats_3_plus_4 = ax.format('Whats {x} + {y}?', x=4, y=3).infer()

print(f'the answer is: {whats_3_plus_4}')

the answer is: {textref_0x136dab050}


You'll notice that above, no inference actually takes place. And accessing the Text object directly to print it will give you a placeholder reference. This is a reference to a graph we've built.

**The graph is only executed when the value is asked for.**

If we want the actual value, we can get the value, and force graph execution and retreive our result.

In [3]:
answer = whats_3_plus_4.value()
print(answer)

Output()

together_text: 17 toks -> meta-llama/Llama-3-8b-chat-hf -> 8 toks (in 0.66s)

4 + 3 = 7


We can actually inspect this graph we've produced, to understand exactly what the agent is doing.

In [4]:
whats_3_plus_4.print_graph()

        _immediate1         
┌───────┬──────────────────┐
│ value │ Whats {x} + {y}? │
└───────┴──────────────────┘

 _immediate2 
┌───────┬───┐
│ value │ 4 │
└───────┴───┘

 _immediate3 
┌───────┬───┐
│ value │ 3 │
└───────┴───┘

_format4
├── ╭─ template: _immediate1 ─╮
│   │ Whats {x} + {y}?        │
│   ╰─────────────────────────╯
├── ╭─ {x}: _immediate2 ─╮
│   │ 4                  │
│   ╰────────────────────╯
└── ╭─ {y}: _immediate3 ─╮
    │ 3                  │
    ╰────────────────────╯

       _immediate5        
┌───────┬────────────────┐
│ value │ <Empty String> │
└───────┴────────────────┘

                _history_complete6                 
┌─────────────────┬───────────────────────────────┐
│ llmMaxTokens    │ 1024                          │
├─────────────────┼───────────────────────────────┤
│ llmModelName    │ meta-llama/Llama-3-8b-chat-hf │
├─────────────────┼───────────────────────────────┤
│ llmProviderName │ together_text                 │
├─────────────────┼───────────────────────────────┤
│ llmTemperature  │ 0.5                           │
└─────────────────┴───────────────────────────────┘
├── user_message: _format4
│   ├── ╭─ template: _immediate1 ─╮
│   │   │ Whats {x} + {y}?        │
│   │   ╰─────────────────────────╯
│   ├── ╭─ {x}: _immediate2 ─╮
│   │   │ 4                  │
│   │   ╰────────────────────╯
│   └── ╭─ {y}: _immediate3 ─╮
│       │ 3                  │
│       ╰────────────────────╯
└── ╭─ system_prompt: _immediate5 ─╮
    │ <Empty String>               │
    ╰──────────────────────────────╯

In [5]:
import axiomic.core.modules as modules


cases = [
    ('how can I help you?', '"Helpful"'),
    ('Get out of my way.', '"Unhelpful"')
]

switch = modules.Switch(cases)

w = ax.Text('How are you today?', name='my_input')

case = switch.infer(w)

case.print_graph()

   default:default:my_input   
┌───────┬────────────────────┐
│ value │ How are you today? │
└───────┴────────────────────┘

          _immediate12           
┌───────┬───────────────────────┐
│ value │ {textref_0x139a4aa20} │
└───────┴───────────────────────┘

              _immediate10               
┌───────┬───────────────────────────────┐
│ value │ You must follow this pattern: │
│       │                               │
│       │ {pattern_desc}                │
│       │                               │
│       │ {textref_0x139a26cf0}         │
│       │ {schema}                      │
│       │                               │
└───────┴───────────────────────────────┘

                               _immediate9                                
┌───────┬────────────────────────────────────────────────────────────────┐
│ value │ Your response must be a valid JSON object. It must             │
│       │ match the this jsonschema exactly.                             │
│       │ Do not include any other text or explanation in your response. │
│       │                                                                │
└───────┴────────────────────────────────────────────────────────────────┘

_format11
├── ╭─ template: _immediate10 ──────╮
│   │ You must follow this pattern: │
│   │                               │
│   │ {pattern_desc}                │
│   │                               │
│   │ {textref_0x139a26cf0}         │
│   │ {schema}                      │
│   │                               │
│   ╰───────────────────────────────╯
└── ╭─ {textref_0x139a26cf0}: _immediate9 ───────────────────────────╮
    │ Your response must be a valid JSON object. It must             │
    │ match the this jsonschema exactly.                             │
    │ Do not include any other text or explanation in your response. │
    │                                                                │
    ╰────────────────────────────────────────────────────────────────╯

...

        _builtin_unary_check25        
┌────────────────┬───────────────────┐
│ unaryCheckName │ CHECK_JSON_SYNTAX │
└────────────────┴───────────────────┘
└── input: _history_complete24
    ├── ╭─ user_message: default:default:my_input ─╮
    │   │ How are you today?                       │
    │   ╰──────────────────────────────────────────╯
    ├── system_prompt: _format16
    │   ├── template: _format13
    │   │   ├── ╭─ template: _immediate12 ─╮
    │   │   │   │ {textref_0x139a4aa20}    │
    │   │   │   ╰──────────────────────────╯
    │   │   │   └── ...
    │   │   └── {textref_0x139a4aa20}: _format11
    │   │       └── ...
    │   ├── ╭─ {pattern_desc}: _immediate14 ───────────────────────────────╮
    │   │   │ Given text, state which of these cases most closely applies: │
    │   │   │ "Helpful"                                                    │
    │   │   │ "Unhelpful"                                                  │
    │   │   │                                                              │
    │   │   ╰──────────────────────────────────────────────────────────────╯
    │   └── ╭─ {schema}: _immediate15 ─────────────────────────────╮
    │       │ {"type": "string", "enum": ["Unhelpful", "Helpful"]} │
    │       ╰──────────────────────────────────────────────────────╯
    ├── ╭─ user_message_0: _immediate20 ─╮
    │   │ how can I help you?            │
    │   ╰────────────────────────────────╯
    ├── ╭─ assistant_message_0: _immediate21 ─╮
    │   │ "Helpful"                           │
    │   ╰─────────────────────────────────────╯
    ├── ╭─ user_message_1: _immediate22 ─╮
    │   │ Get out of my way.             │
    │   ╰────────────────────────────────╯
    └── ╭─ assistant_message_1: _immediate23 ─╮
        │ "Unhelpful"                         │
        ╰─────────────────────────────────────╯

_boolean_gate26
├── value: _history_complete24
│   ├── ╭─ user_message: default:default:my_input ─╮
│   │   │ How are you today?                       │
│   │   ╰──────────────────────────────────────────╯
│   ├── system_prompt: _format16
│   │   ├── template: _format13
│   │   │   ├── ╭─ template: _immediate12 ─╮
│   │   │   │   │ {textref_0x139a4aa20}    │
│   │   │   │   ╰──────────────────────────╯
│   │   │   │   └── ...
│   │   │   └── {textref_0x139a4aa20}: _format11
│   │   │       └── ...
│   │   ├── ╭─ {pattern_desc}: _immediate14 ───────────────────────────────╮
│   │   │   │ Given text, state which of these cases most closely applies: │
│   │   │   │ "Helpful"                                                    │
│   │   │   │ "Unhelpful"                                                  │
│   │   │   │                                                              │
│   │   │   ╰──────────────────────────────────────────────────────────────╯
│   │   └── ╭─ {schema}: _immediate15 ─────────────────────────────╮
│   │       │ {"type": "string", "enum": ["Unhelpful", "Helpful"]} │
│   │       ╰──────────────────────────────────────────────────────╯
│   ├── ╭─ user_message_0: _immediate20 ─╮
│   │   │ how can I help you?            │
│   │   ╰────────────────────────────────╯
│   ├── ╭─ assistant_message_0: _immediate21 ─╮
│   │   │ "Helpful"                           │
│   │   ╰─────────────────────────────────────╯
│   ├── ╭─ user_message_1: _immediate22 ─╮
│   │   │ Get out of my way.             │
│   │   ╰────────────────────────────────╯
│   └── ╭─ assistant_message_1: _immediate23 ─╮
│       │ "Unhelpful"                         │
│       ╰─────────────────────────────────────╯
└── gating: _builtin_unary_check25
    └── input: _history_complete24
        ├── ╭─ user_message: default:default:my_input ─╮
        │   │ How are you today?                       │
        │   ╰──────────────────────────────────────────╯
        ├── system_prompt: _format16
        │   ├── template: _format13
        │   │   └── ...
        │   ├── ╭─ {pattern_desc}: _immediate14 ───────────────────────────────╮
        │   │   │ Given text, state which of these cases most closely applies: │
        │   │   │ "Helpful"                                                    │
        │   │   │ "Unhelpful"                                                  │
        │   │   │                                                              │
        │   │   ╰──────────────────────────────────────────────────────────────╯
        │   │   └── ...
        │   └── ╭─ {schema}: _immediate15 ─────────────────────────────╮
        │       │ {"type": "string", "enum": ["Unhelpful", "Helpful"]} │
        │       ╰──────────────────────────────────────────────────────╯
        │       └── ...
        ├── ╭─ user_message_0: _immediate20 ─╮
        │   │ how can I help you?            │
        │   ╰────────────────────────────────╯
        ├── ╭─ assistant_message_0: _immediate21 ─╮
        │   │ "Helpful"                           │
        │   ╰─────────────────────────────────────╯
        ├── ╭─ user_message_1: _immediate22 ─╮
        │   │ Get out of my way.             │
        │   ╰────────────────────────────────╯
        └── ╭─ assistant_message_1: _immediate23 ─╮
            │ "Unhelpful"                         │
            ╰─────────────────────────────────────╯

                          _immediate27                          
┌───────┬──────────────────────────────────────────────────────┐
│ value │ {"type": "string", "enum": ["Unhelpful", "Helpful"]} │
└───────┴──────────────────────────────────────────────────────┘

       _builtin_binary_check28        
┌─────────────────┬──────────────────┐
│ binaryCheckName │ CHECK_JSONSCHEMA │
└─────────────────┴──────────────────┘
├── input_main: _boolean_gate26
│   ├── value: _history_complete24
│   │   ├── ╭─ user_message: default:default:my_input ─╮
│   │   │   │ How are you today?                       │
│   │   │   ╰──────────────────────────────────────────╯
│   │   ├── system_prompt: _format16
│   │   │   ├── template: _format13
│   │   │   │   └── ...
│   │   │   ├── ╭─ {pattern_desc}: _immediate14 ───────────────────────────────╮
│   │   │   │   │ Given text, state which of these cases most closely applies: │
│   │   │   │   │ "Helpful"                                                    │
│   │   │   │   │ "Unhelpful"                                                  │
│   │   │   │   │                                                              │
│   │   │   │   ╰──────────────────────────────────────────────────────────────╯
│   │   │   │   └── ...
│   │   │   └── ╭─ {schema}: _immediate15 ─────────────────────────────╮
│   │   │       │ {"type": "string", "enum": ["Unhelpful", "Helpful"]} │
│   │   │       ╰──────────────────────────────────────────────────────╯
│   │   │       └── ...
│   │   ├── ╭─ user_message_0: _immediate20 ─╮
│   │   │   │ how can I help you?            │
│   │   │   ╰────────────────────────────────╯
│   │   ├── ╭─ assistant_message_0: _immediate21 ─╮
│   │   │   │ "Helpful"                           │
│   │   │   ╰─────────────────────────────────────╯
│   │   ├── ╭─ user_message_1: _immediate22 ─╮
│   │   │   │ Get out of my way.             │
│   │   │   ╰────────────────────────────────╯
│   │   └── ╭─ assistant_message_1: _immediate23 ─╮
│   │       │ "Unhelpful"                         │
│   │       ╰─────────────────────────────────────╯
│   └── gating: _builtin_unary_check25
│       └── input: _history_complete24
│           ├── ╭─ user_message: default:default:my_input ─╮
│           │   │ How are you today?                       │
│           │   ╰──────────────────────────────────────────╯
│           │   └── ...
│           ├── system_prompt: _format16
│           │   └── ...
│           ├── ╭─ user_message_0: _immediate20 ─╮
│           │   │ how can I help you?            │
│           │   ╰────────────────────────────────╯
│           │   └── ...
│           ├── ╭─ assistant_message_0: _immediate21 ─╮
│           │   │ "Helpful"                           │
│           │   ╰─────────────────────────────────────╯
│           │   └── ...
│           ├── ╭─ user_message_1: _immediate22 ─╮
│           │   │ Get out of my way.             │
│           │   ╰────────────────────────────────╯
│           │   └── ...
│           └── ╭─ assistant_message_1: _immediate23 ─╮
│               │ "Unhelpful"                         │
│               ╰─────────────────────────────────────╯
│               └── ...
└── ╭─ input_secondary: _immediate27 ──────────────────────╮
    │ {"type": "string", "enum": ["Unhelpful", "Helpful"]} │
    ╰──────────────────────────────────────────────────────╯

_boolean_gate29
├── value: _boolean_gate26
│   ├── value: _history_complete24
│   │   ├── ╭─ user_message: default:default:my_input ─╮
│   │   │   │ How are you today?                       │
│   │   │   ╰──────────────────────────────────────────╯
│   │   ├── system_prompt: _format16
│   │   │   ├── template: _format13
│   │   │   │   └── ...
│   │   │   ├── ╭─ {pattern_desc}: _immediate14 ───────────────────────────────╮
│   │   │   │   │ Given text, state which of these cases most closely applies: │
│   │   │   │   │ "Helpful"                                                    │
│   │   │   │   │ "Unhelpful"                                                  │
│   │   │   │   │                                                              │
│   │   │   │   ╰──────────────────────────────────────────────────────────────╯
│   │   │   │   └── ...
│   │   │   └── ╭─ {schema}: _immediate15 ─────────────────────────────╮
│   │   │       │ {"type": "string", "enum": ["Unhelpful", "Helpful"]} │
│   │   │       ╰──────────────────────────────────────────────────────╯
│   │   │       └── ...
│   │   ├── ╭─ user_message_0: _immediate20 ─╮
│   │   │   │ how can I help you?            │
│   │   │   ╰────────────────────────────────╯
│   │   ├── ╭─ assistant_message_0: _immediate21 ─╮
│   │   │   │ "Helpful"                           │
│   │   │   ╰─────────────────────────────────────╯
│   │   ├── ╭─ user_message_1: _immediate22 ─╮
│   │   │   │ Get out of my way.             │
│   │   │   ╰────────────────────────────────╯
│   │   └── ╭─ assistant_message_1: _immediate23 ─╮
│   │       │ "Unhelpful"                         │
│   │       ╰─────────────────────────────────────╯
│   └── gating: _builtin_unary_check25
│       └── input: _history_complete24
│           ├── ╭─ user_message: default:default:my_input ─╮
│           │   │ How are you today?                       │
│           │   ╰──────────────────────────────────────────╯
│           │   └── ...
│           ├── system_prompt: _format16
│           │   └── ...
│           ├── ╭─ user_message_0: _immediate20 ─╮
│           │   │ how can I help you?            │
│           │   ╰────────────────────────────────╯
│           │   └── ...
│           ├── ╭─ assistant_message_0: _immediate21 ─╮
│           │   │ "Helpful"                           │
│           │   ╰─────────────────────────────────────╯
│           │   └── ...
│           ├── ╭─ user_message_1: _immediate22 ─╮
│           │   │ Get out of my way.             │
│           │   ╰────────────────────────────────╯
│           │   └── ...
│           └── ╭─ assistant_message_1: _immediate23 ─╮
│               │ "Unhelpful"                         │
│               ╰─────────────────────────────────────╯
│               └── ...
└── gating: _builtin_binary_check28
    ├── input_main: _boolean_gate26
    │   ├── value: _history_complete24
    │   │   ├── ╭─ user_message: default:default:my_input ─╮
    │   │   │   │ How are you today?                       │
    │   │   │   ╰──────────────────────────────────────────╯
    │   │   │   └── ...
    │   │   ├── system_prompt: _format16
    │   │   │   └── ...
    │   │   ├── ╭─ user_message_0: _immediate20 ─╮
    │   │   │   │ how can I help you?            │
    │   │   │   ╰────────────────────────────────╯
    │   │   │   └── ...
    │   │   ├── ╭─ assistant_message_0: _immediate21 ─╮
    │   │   │   │ "Helpful"                           │
    │   │   │   ╰─────────────────────────────────────╯
    │   │   │   └── ...
    │   │   ├── ╭─ user_message_1: _immediate22 ─╮
    │   │   │   │ Get out of my way.             │
    │   │   │   ╰────────────────────────────────╯
    │   │   │   └── ...
    │   │   └── ╭─ assistant_message_1: _immediate23 ─╮
    │   │       │ "Unhelpful"                         │
    │   │       ╰─────────────────────────────────────╯
    │   │       └── ...
    │   └── gating: _builtin_unary_check25
    │       └── input: _history_complete24
    │       